### Real, Complex and Symplectic Reflection Groups - March 2023, RUB

## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 3. Enumerating Complex Reflection Groups

## Setup

First, reload the algorithms from the last day ...

In [ ]:
LoadPackage("jupyterviz");
opts:= rec(vertexwidth:= 12, vertexheight:= 12, edgecolor:= "#def");;
Read("orbits.g");
Read("variants.g");
Read("examples.g");

## Complex Reflection Groups ...

... don't have
* well-behaved length
* well-defined generators
* root systems
* ...

But they do have
* parabolic subgroups
* ...
* a nice (Coxeter-like) presentation

## Data Nodes

* Recall the modified orbit algorithms.
* Here is all in one:  words, transversal and edges.

In [ ]:
orbit_and_more := function(aaa, x, under)
    local   list,  words,  perms,  edges,  i,  k,  z,  l;
    list:= [x];  words:= [[]];  perms:= [aaa[1]^0];  edges:= [];
    i:= 0;
    while i < Length(list) do
        i:= i+1;
        for k in [1..Length(aaa)] do
            z:= under(list[i], aaa[k]);
            l:= Position(list, z);
            if l = fail then
                Add(list, z);
                Add(words, onWords(words[i], k));
                Add(perms, perms[i] * aaa[k]);
                Add(edges, [i, Length(list)]);
            else
                Add(edges, [i, l]);
            fi;
        od;
    od;
    return rec(list:= list, words:= words, perms:= perms, edges:= edges);
end;

* for example

In [ ]:
orb:= orbit_and_more(transpositions(5), [1,2], OnSets);;
orb.list;
orb.perms;
orb.words;
edges:= Filtered(Set(orb.edges), x-> x[1] <> x[2]);;
PlotGraph(edges, opts);

* managing all these property lists in parallel is a bit unwieldy, and perhaps confusing in the long run ...
* let's transpose the setup and store properties as data with each node.

##  Data Nodes

### Data Type

In [ ]:
ItemFamily := NewFamily("ItemFamily", IsObject);

DeclareRepresentation("IsItem",
    IsComponentObjectRep and IsAttributeStoringRep,
    ["key", "idx", "data", "next"]
);

ItemType := NewType(ItemFamily, IsItem);

### Constructor

In [ ]:
Item := function(key)
    local   r;
    r := rec(key := key, data := rec(), next := []);
    return Objectify(ItemType, r);
end;

In [ ]:
item:= Item([1,2]);

### Printing

In [ ]:
InstallMethod(PrintObj, "for items", true, [IsItem], 0, function(item)
    Print("Item( ", item!.key, " )");
end);

InstallMethod(String, "for items", true, [IsItem], 0, function(item)
    return Concatenation("Item( ", item!.key, " )");
end);

In [ ]:
item;

###  Comparison

In [ ]:
InstallMethod(\=, "for items", true, [IsItem, IsItem], 0, function(itemL, itemR)
    return itemL!.key = itemR!.key;
end);

InstallMethod(\<, "for items", true, [IsItem, IsItem], 0, function(itemL, itemR)
    return itemL!.key < itemR!.key;
end);

In [ ]:
item = item;
item < item;
item <= item;
Set([item, item]);

### Data Orbits

In [ ]:
orbit_with_data := function(aaa, item, under)
    local   list,  x,  k,  a,  y,  z;
    list := [item];  item!.idx:= 1;  
    item!.data:= rec(perm:= (), word:= []);
    for x in list do
        for k in [1..Length(aaa)] do
            a:= aaa[k];
            y := Item(under(x!.key, a));
            z := First(list, z -> z = y);
            if z = fail then
                Add(list, y);  y!.idx := Length(list);
                y!.data := rec(  
                  perm := x!.data.perm * a,
                  word := onWords(x!.data.word, k),
                );
                x!.next[k] := y!.idx;
            else
                x!.next[k] := z!.idx;
            fi;
        od;
    od;
    return list;
end;

In [ ]:
orb:= orbit_with_data(transpositions(5), item, OnSets);

In [ ]:
List(orb, x-> x!.idx);
List(orb, x-> x!.data.perm);
List(orb, x-> x!.data.word);
edges:= Union(List(orb, o-> List(o!.next, t-> [o!.idx, t])));;
edges:= Filtered(edges, x-> x[1] <> x[2]);;
PlotGraph(edges, opts);

## Smart Nodes

In [ ]:
NodeFamily := NewFamily("NodeFamily", IsObject);

DeclareRepresentation("IsNode",
    IsComponentObjectRep and IsAttributeStoringRep,
["idx", "word", "next"]
);

NodeType := NewType(NodeFamily, IsNode);

In [ ]:
Node := function(word, data)
    local   node;
    node := Objectify(NodeType, rec(word := word, data := data, next := []));
    Add(data.list, node);  node!.idx := Length(data.list);
    data.active := data.active + 1;
    return node;
end;

### Print

In [ ]:
InstallMethod(PrintObj, "for nodes", true, [IsNode], 0, function(node)
    Print("Node( ", node!.idx, " )");
end);

In [ ]:
InstallMethod(String, "for nodes", true, [IsNode], 0, function(node)
    return Concatenation("Node( ", String(node!.idx), " )");
end);

In [ ]:
data := rec(list := [], active := 0);

In [ ]:
node := Node([], data);

In [ ]:
String(node);

### Comparison

In [ ]:
InstallMethod(\=, "for nodes", true, [IsNode, IsNode], 0, function(nodeL, nodeR)
    return nodeL!.idx = nodeR!.idx;
end);

In [ ]:
node = node;

In [ ]:
InstallMethod(\<, "for nodes", true, [IsNode, IsNode], 0, function(nodeL, nodeR)
    return nodeL!.idx < nodeR!.idx;
end);

In [ ]:
node < node;

In [ ]:
node <= node;

In [ ]:
node > node;

## Enumeration

* Suppose that a group $G$ is given by a **presentation** $\langle X \mid R \rangle$, consisting of a (finite) set $X$ of abstract **generators** $x_1, x_2, \dots$, and a (finite) list $R$ of **relations** $l_j = r_j$, where both $l_j$ and $r_j$ are words in $X \cup X^{-1}$.

* For convenience, we assume that $X$ is closed under inverses: $X  = X^{-1}$.

###  Flatness

In [ ]:
isActive := node -> not IsBound(node!.flat);

In [ ]:
isActive(node);

In [ ]:
node!.flat := 0;;
isActive(node);

In [ ]:
Unbind(node!.flat);
isActive(node);

In [ ]:
flat := function(node)
    while IsBound(node!.flat) do  node := node!.flat;  od;
    return node;
end;

### Images

In [ ]:
getImage := function(node, s)
    if s < 0 then  s:= node!.data.invr[-s];  fi;
    return GetWithDefault(node!.next, s, false);
end;

In [ ]:
sprout := function(node, s)
    local   next;
    next:= Node(onWords(node!.word, s), node!.data);
    node!.next[s] := next;
    next!.next[node!.data.invr[s]] := node;
    return next;
end;

### Actions

In [ ]:
onNodesPartial := function(node, s)
    local   next;
    next := getImage(node, s);
    if next = false then  return false;  fi;
    return flat(next);
end;

In [ ]:
onNodesSprout := function(node, s)
    local   next;
    next := getImage(node, s);
    if next = false then 
        return sprout(node, s);
    else
        return flat(next);  
    fi;
end;

In [ ]:
nodeUnderWordSprout := function(node, word)
    local   s;
    for s in word do
        node := onNodesSprout(node, s);
    od;
    return node;
end;

In [ ]:
nodeUnderWordPartial := function(node, word)
    local   s;
    for s in word do
        node := onNodesPartial(node, s);
        if node = false then  return node;  fi;
    od;
    return node;
end;

### Edges

In [ ]:
setImage := function(node, s, next)
    local   pair;
    if IsBound(node!.next[s]) then
        pair := Set(List([next, node!.next[s]], flat));
        if Length(pair) = 2 then           # coincidence: stack!
            mergeNodes(pair[2], pair[1]);
        fi;
    else
        node!.next[s] := next;           # deduction!
    fi;
end;

In [ ]:
mergeNodes := function(node, other)
    local   s;
    node!.flat:= other;
    Print(node, " -> ", other, "\n");
    for s in PositionsBound(node!.next) do
        updateEdge(other, s, node!.next[s]);
    od;
    node!.data.active:= node!.data.active - 1;
end;

In [ ]:
updateEdge := function(node, s, next)
    setImage(node, s, next);
    setImage(next, node!.data.invr[s], node);
end;

###  Tracing Relations

* node!word = word

In [ ]:
trace := function(node, word)
    local   other;
    other := nodeUnderWordSprout(node, word{[1..Length(word)-1]});
    updateEdge(other, word[Length(word)], node);
end;

In [ ]:
process := function(node, s)
    local   variant,  next;
    for variant in node!.data.variants[s] do
        if isActive(node) then
            next:= nodeUnderWordPartial(node, variant);
            if next <> false then  updateEdge(node, s, next);  fi;
        fi;
    od;
    #    if isActive(node) then  trace(node, [s, node!.data.invr[s]]);  fi;
    if isActive(node) and not IsBound(node!.next[s]) then sprout(node, s); fi;
end;

In [ ]:
enumerate := function(genrel)
    local  data,  node,  word,  s;

    # initialize.
    data := rec(list := [], active := 0);
    data.invr := genrel.invr;
    data.variants:= VariantsRelations(genrel);
    node := Node([], data);
    
    # first close the subgroup tables.
    for word in genrel.sbgp do
        trace(node, word);
    od;
    
    # process nodes in the queue
    for node in data.list do
        for s in genrel.gens do
            process(node, s);
        od;
    od;

    # return data
    return data;
end;

In [ ]:
G := G10;

In [ ]:
data:= enumerate(G);

In [ ]:
data.active;

In [ ]:
gens:= List(TransposedMat(List(data.list, x-> List(x!.next, y-> y!.idx))), PermList);

In [ ]:
sizeOfGroup(GroupWithGenerators(gens));

* Next: extract and plot the graph!

In [ ]:
edges:= Union(List(data.list, node -> List(node!.next, x -> [node!.idx, x!.idx])));;
edges:= Filtered(edges, x -> x[1] <> x[2]);

In [ ]:
PlotGraph(edges, opts);

##  Exercises, etc.

* Find matrices for the reflection representation of a given complex reflection group $G$.
* Find a way to enumerate the (conjugacy classes of) parabolic subgroups $P$ of a complex reflection group $G$.
* Compute the normalizer of parabolic subgroup $P$ in $G$.  Does $P$ always have a complement $H$? If so, does $H$ have a natural set of generators? 